### ЗАДАЧА: Контроль доставок и просрочек

Логистическая компания ведёт учёт отправок по складам.
Нужно обработать файл с доставками, найти просрочки, дубликаты и сформировать отчёт для менеджера.

НЕОБХОДИМО РЕАЛИЗОВАТЬ:

1. Сохранить исходный список строк в файл delivery_log.txt.

2. Прочитать файл delivery_log.txt через with open и загрузить записи в список словарей.

3. Для каждого склада посчитать:
   - общее количество отправок
   - количество просроченных отправок (actual_day > planned_day).

4. Собрать множество всех городов доставки.

5. Найти shipment_id, которые встречаются больше одного раза (дубликаты).

6. Для каждого города посчитать суммарное количество доставленных единиц товара (только status = delivered).

7. Найти склад с самым высоким процентом просрочек.

8. Записать итоговый отчёт в файл delivery_report.txt (статистика складов, города, дубликаты, топ-склад по просрочкам, объёмы по городам).

In [4]:
lines = [
    "SHP-1001,North,Chicago,3,3,40,delivered",
    "SHP-1002,North,Boston,4,6,20,delivered",
    "SHP-1003,East,Miami,2,2,35,cancelled",
    "SHP-1004,East,Chicago,5,8,15,delivered",
    "SHP-1005,West,Dallas,1,1,50,delivered",
    "SHP-1006,West,Miami,2,4,30,delivered",
    "SHP-1002,North,Boston,4,6,20,delivered",
    "SHP-1007,South,Atlanta,6,6,25,in_transit",
    "SHP-1008,South,Dallas,3,5,10,delivered"
]

deliveries = []
# список словарей с записями доставок
# ключи: shipment_id, warehouse, city, planned_day, actual_day, items, status

warehouse_stats = {}
# warehouse -> {"total": 0, "delayed": 0}

cities = set()
# множество городов доставки

shipment_counter = {}
# shipment_id -> количество встреч

duplicate_shipments = []
# список shipment_id, которые дублируются

city_delivered_items = {}
# city -> суммарное количество items для status == delivered


with open("delivery_log.txt", "w", encoding="utf-8") as file:
    for line in lines:
        file.write(line + "\n")

with open("delivery_log.txt", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        parts = line.split(",")
        shipment_id, warehouse, city = parts[0], parts[1], parts[2]
        planned_day = int(parts[3])
        actual_day = int(parts[4])
        items = int(parts[5])
        status = parts[6]

        deliveries.append({
            "shipment_id": shipment_id,
            "warehouse": warehouse,
            "city": city,
            "planned_day": planned_day,
            "actual_day": actual_day,
            "items": items,
            "status": status
        })
print(deliveries)

for delivery in deliveries:
    shipment_id, warehouse, city, planned_day, actual_day, items, status = delivery["shipment_id"], delivery["warehouse"], delivery["city"], delivery["planned_day"], delivery["actual_day"], delivery["items"], delivery["status"]

# 1
    if warehouse not in warehouse_stats:
        warehouse_stats[warehouse] = {"total":0, "delayed":0}
    warehouse_stats[warehouse]["total"] += 1
# 2   
    if actual_day > planned_day:
        warehouse_stats[warehouse]["delayed"] += 1
# 3   
    cities.add(city)
# 4
    if shipment_id in shipment_counter:
        shipment_counter[shipment_id] += 1
    else:
        shipment_counter[shipment_id] = 1
# 5
    if status == "delivered":
        if city in city_delivered_items:
            city_delivered_items[city] += items
        else:
            city_delivered_items[city] = items
# 6
for shipment_id, count in shipment_counter.items():
    if count > 1:
        duplicate_shipments.append(shipment_id)
# 7
worst_warehouse = None
max_delay_rate = -1

for warehouse, stats in warehouse_stats.items():
    delay_rate = stats["delayed"] / stats["total"]
    if delay_rate > max_delay_rate:
        max_delay_rate = delay_rate
        worst_warehouse = warehouse

# 8
with open("delivery_report.txt", "w", encoding="utf-8") as file:
    file.write("Отчет по доставкам\n")
    file.write("=" * 50 + "\n\n")

    file.write("\nСтатистика по складам:\n")
    for warehouse, stats in warehouse_stats.items():
        print(f"{warehouse}: total={stats["total"]}, delayed={stats["delayed"]}")

    file.write("\nСписок городов:\n")
    file.write(",".join(cities))

    file.write("\nДубликаты:\n")
    if duplicate_shipments:
        file.write(",".join(duplicate_shipments) + "\n")
    else:
        print("Нет\n")

    file.write("\nСклад с макимальным процентом просрочек:\n")
    file.write(f"{worst_warehouse} ({max_delay_rate})\n")

    file.write("\nДоставленные объёмы по городам:\n")
    for city, total_items in city_delivered_items.items():
        file.write(f"{city}: {total_items}\n")





[{'shipment_id': 'SHP-1001', 'warehouse': 'North', 'city': 'Chicago', 'planned_day': 3, 'actual_day': 3, 'items': 40, 'status': 'delivered'}, {'shipment_id': 'SHP-1002', 'warehouse': 'North', 'city': 'Boston', 'planned_day': 4, 'actual_day': 6, 'items': 20, 'status': 'delivered'}, {'shipment_id': 'SHP-1003', 'warehouse': 'East', 'city': 'Miami', 'planned_day': 2, 'actual_day': 2, 'items': 35, 'status': 'cancelled'}, {'shipment_id': 'SHP-1004', 'warehouse': 'East', 'city': 'Chicago', 'planned_day': 5, 'actual_day': 8, 'items': 15, 'status': 'delivered'}, {'shipment_id': 'SHP-1005', 'warehouse': 'West', 'city': 'Dallas', 'planned_day': 1, 'actual_day': 1, 'items': 50, 'status': 'delivered'}, {'shipment_id': 'SHP-1006', 'warehouse': 'West', 'city': 'Miami', 'planned_day': 2, 'actual_day': 4, 'items': 30, 'status': 'delivered'}, {'shipment_id': 'SHP-1002', 'warehouse': 'North', 'city': 'Boston', 'planned_day': 4, 'actual_day': 6, 'items': 20, 'status': 'delivered'}, {'shipment_id': 'SHP-10